# **TMBD MOVIE DATA ANALYSIS**

This project extracts movie data from the TMDB API, cleans and preprocesses the dataset, and performs in-depth exploratory data analysis. The goal is to uncover meaningful insights, identify trends, and support data-driven decision-making based on movie performance, audience behavior, and industry patterns.

## **Data Extraction**
Extracts movie data from the TMDB API

In [1]:
#Imports packages and modules

#Import os and sys
import os
import sys 
from pathlib import Path

#Extarcts the root path of the project and appends it to the sys path
project_root = Path().resolve().parent
sys.path.append(str(project_root))

#Imports loadEnv from config module
from Config.config import loadEnv, getURL, create_retry
#Imports extractDataFromAPI from the extract data module
from Data_Extraction.extractData import extractDataFromAPI

In [2]:
movie_ids = [0, 299534,19995,140607,299536,597,135397, 420818, 24428, 168259, 99861,
                    284054, 12445, 181808, 330457, 351286, 109445, 321612, 260513] 

API_KEY = loadEnv(fileName="API_KEY")
url = getURL()
session = create_retry()
data = extractDataFromAPI(session = session, url = url, API_KEY = API_KEY, movie_ids=movie_ids)


In [3]:
#Outputs the columns if the dataFrame is not None or empty
data.columns if data is not None and not data.empty else "No data extracted from the API"

Index(['adult', 'backdrop_path', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'origin_country', 'original_language', 'original_title',
       'overview', 'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'belongs_to_collection.id',
       'belongs_to_collection.name', 'belongs_to_collection.poster_path',
       'belongs_to_collection.backdrop_path', 'belongs_to_collection'],
      dtype='object')

In [4]:
data.head()

,adult,backdrop_path,budget,genres,homepage,id,imdb_id,origin_country,original_language,original_title,...,tagline,title,video,vote_average,vote_count,belongs_to_collection.id,belongs_to_collection.name,belongs_to_collection.poster_path,belongs_to_collection.backdrop_path,belongs_to_collection
0,False,/9wXPKruA6bWYk2co5ix6fH59Qr8.jpg,356000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 878, ...",https://www.marvel.com/movies/avengers-endgame,299534,tt4154796,[US],en,Avengers: Endgame,...,Avenge the fallen.,Avengers: Endgame,False,8.237,26974,86311.0,The Avengers Collection,/yFSIUVTCvgYrpalUktulvk3Gi5Y.jpg,/zuW6fOiusv4X9nnW3paHGfXcSll.jpg,NaN
1,False,/7JNzw1tSZZEgsBw6lu0VfO2X2Ef.jpg,237000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",https://www.avatar.com/movies/avatar,19995,tt0499549,[US],en,Avatar,...,Enter the world of Pandora.,Avatar,False,7.594,32878,87096.0,Avatar Collection,/3C5brXxnBxfkeKWwA1Fh4xvy4wr.jpg,/6qkJLRCZp9Y3ovXti5tSuhH0DpO.jpg,NaN
2,False,/8BTsTfln4jlQrLXUBquXJ0ASQy9.jpg,245000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",http://www.starwars.com/films/star-wars-episod...,140607,tt2488496,[US],en,Star Wars: The Force Awakens,...,Every generation has a story.,Star Wars: The Force Awakens,False,7.300,20104,10.0,Star Wars Collection,/22dj38IckjzEEUZwN1tPU5VJ1qq.jpg,/qVPChlozQ1BP3svfHjiAdNneMGA.jpg,NaN
3,False,/mDfJG3LC3Dqb67AZ52x3Z0jU0uB.jpg,300000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",https://www.marvel.com/movies/avengers-infinit...,299536,tt4154756,[US],en,Avengers: Infinity War,...,Destiny arrives all the same.,Avengers: Infinity War,False,8.235,31187,86311.0,The Avengers Collection,/yFSIUVTCvgYrpalUktulvk3Gi5Y.jpg,/zuW6fOiusv4X9nnW3paHGfXcSll.jpg,NaN
4,False,/xnHVX37XZEp33hhCbYlQFq7ux1J.jpg,200000000,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",https://www.paramountmovies.com/movies/titanic,597,tt0120338,[US],en,Titanic,...,Nothing on earth could come between them.,Titanic,False,7.903,26519,NaN,NaN,NaN,NaN,NaN
